<a href="https://colab.research.google.com/github/blindTissue/nlp_hw7/blob/main/davinci.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

from datasets import load_dataset, concatenate_datasets

dataset = load_dataset("boolq")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
train_dataset = dataset["train"]
true_instances = train_dataset.filter(lambda x: x['answer'] == True)
false_instances = train_dataset.filter(lambda x: x['answer'] == False)
print(len(true_instances))
print(len(false_instances))


5874
3553


In [ ]:
import random

k = true_instances[0,1]
print(k)

{'question': ['do iran and afghanistan speak the same language', 'do good samaritan laws protect those who help at an accident'], 'answer': [True, True], 'passage': ['Persian (/ˈpɜːrʒən, -ʃən/), also known by its endonym Farsi (فارسی fārsi (fɒːɾˈsiː) ( listen)), is one of the Western Iranian languages within the Indo-Iranian branch of the Indo-European language family. It is primarily spoken in Iran, Afghanistan (officially known as Dari since 1958), and Tajikistan (officially known as Tajiki since the Soviet era), and some other regions which historically were Persianate societies and considered part of Greater Iran. It is written in the Persian alphabet, a modified variant of the Arabic script, which itself evolved from the Aramaic alphabet.', "Good Samaritan laws offer legal protection to people who give reasonable assistance to those who are, or who they believe to be, injured, ill, in peril, or otherwise incapacitated. The protection is intended to reduce bystanders' hesitation to

In [ ]:
import random


def get_balanced_string(length=8):
    out = ""
    for i in range(length // 2):
        true_item = true_instances[random.randrange(len(true_instances))]
        false_item = false_instances[random.randrange(len(false_instances))]
        out += 'Q: ' + true_item['question'] + '? P: ' + true_item['passage'] + " A: " + str(true_item['answer']) + '\n'
        out += 'Q: ' + false_item['question'] + '? P: ' + false_item['passage'] + " A: " + str(false_item['answer'])+ '\n'
    return out

print(get_balanced_string())

def get_random_query(t):
    if t:
        item = true_instances[random.randrange(len(true_instances))]
    else:
        item = false_instances[random.randrange(len(false_instances))]
    return 'Q: ' + item['question'] + '? P: ' + item['passage']

print(get_random_query(False))

    

Q: is working tax credit classed as a benefit? P: Working Tax Credit (WTC) is a state benefit in the United Kingdom made to people who work and have a low income. It was introduced in April 2003 and is a means-tested benefit. Despite their name, tax credits are not to be confused with tax credits linked to a person's tax bill, because they are used to top-up wages. Unlike most other benefits, it is paid by HM Revenue and Customs (HMRC). A: True
Q: do they have in n out in seattle? P: In-N-Out Burger is an American regional chain of fast food restaurants with locations primarily in the American Southwest and Pacific coast. It was founded in Baldwin Park, California in 1948 by Harry Snyder and Esther Snyder. The chain is currently headquartered in Irvine, California and has slowly expanded outside Southern California into the rest of California, as well as into Arizona, Nevada, Utah, Texas, and Oregon. The current owner is Lynsi Snyder, the Snyders' only grandchild. A: False
Q: is there 

In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import openai

openai.api_key = "something"

shots = get_balanced_string()
p = shots + get_random_query(True)
response = openai.Completion.create(
  model="text-davinci-003",
  prompt=p,
  temperature=0.7,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
)

In [ ]:
if "True" in response["choices"][0]["text"]:
    print("hi")

hi


In [ ]:

correctTrue = 0
correctFalse = 0
for i in range(15):
    t_q = get_random_query(True)
    f_q = get_random_query(False)
    p = shots + t_q

    response = openai.Completion.create(
    model="text-davinci-003",
    prompt=p,
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    )
    if "True" in response["choices"][0]["text"]:
        correctTrue += 1
    else:
        print(t_q)
        print(response["choices"][0]["text"])
    
    p = shots + get_random_query(False)

    response = openai.Completion.create(
    model="text-davinci-003",
    prompt=p,
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    )
    if "False" in response["choices"][0]["text"]:
        correctFalse += 1
    else:
        print(f_q)
        print(response["choices"][0]["text"])
print(shots)
print(correctTrue)
print(correctFalse)


    

Q: what book of the old testament is considered to have the greatest poetry? P: The Old Testament is considered one of the most important and influential works of literature in world literature, and has provided inspiration for writers (both religious and secular) throughout subsequent history. In 1886, Friedrich Nietzsche wrote: ``In the Jewish Old Testament, there are men, things and speeches in so grand a style that Greek and Indian literature have nothing to compare to it. One stands with awe and reverence before these tremendous remnants of what man once was... The taste for the Old Testament is a touchstone of ``greatness'' and ``smallness''. To have glued this New Testament, a kind of rococo of taste in every respect, to the Old Testament to form one book... that is perhaps the greatest audacity and sin against the spirit that Europe has on its conscience.''
 Many consider the Book of Psalms to be the greatest work of poetry in the Old Testament. A: The Book of Psalms
Q: does fi